<a href="https://colab.research.google.com/github/El-Wally/Movie-Recommender/blob/main/MoviesRecommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importing packages**

In [134]:
import pandas as pd


**Importing the dataset**

In [135]:
!wget -O moviedataset.zip https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/moviedataset.zip
print('unziping ...')
!unzip -o -j moviedataset.zip 

--2022-07-09 19:01:00--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/moviedataset.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 160301210 (153M) [application/zip]
Saving to: ‘moviedataset.zip’

moviedataset.zip    100%[===================>] 152.88M  25.4MB/s    in 6.3s    

2022-07-09 19:01:07 (24.2 MB/s) - ‘moviedataset.zip’ saved [160301210/160301210]

unziping ...
Archive:  moviedataset.zip
  inflating: links.csv               
  inflating: movies.csv              
  inflating: ratings.csv             
  inflating: README.txt              
  inflating: tags.csv                


In [136]:
df = pd.read_csv('movies.csv')
df.head(5)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [137]:
df['title'][7001]

'Accident (1967)'

checking for null values

In [138]:
df.isnull().sum()

movieId    0
title      0
genres     0
dtype: int64

In [139]:
df1 = pd.read_csv('tags.csv')
df1.head(5)

,userId,movieId,tag,timestamp
0,19,2324,bittersweet,1428651158
1,19,2324,holocaust,1428651112
2,19,2324,World War II,1428651118
3,23,7075,hilarious,1378675786
4,23,7075,Underrated,1378675786


In [140]:
df1.isnull().sum()

userId        0
movieId       0
tag          16
timestamp     0
dtype: int64

In [141]:
df1.shape[0]

586994

In [169]:
(df1['movieId']==72).sum()

31

In [154]:
df1['tag'][5]

'Garath'

In [178]:
for i in range (70,71):
  dff1 = df1[df1['movieId'].values  == i]['tag']
  print(dff1)
  dff1=dff1.tolist()
  dff1 = list(dict.fromkeys(dff1))
  df1.drop(df1.index[i:i+len(dff1)])
  df1['tag'][i] = dff1
  print(i)
 

  

 

49        [lesbian, nudity (topless), circus, Gay, Relig...
652                                                  btaege
3989                                              cult film
3990                                      Quentin Tarantino
3991                                           twist ending
                                ...                        
580457                                          Salma Hayek
580458                                            strippers
580459                                            Tarantino
580460                                             vampires
583825                                        Didn't finish
Name: tag, Length: 366, dtype: object


TypeError: ignored

548

In [ ]:
df.join(df1,on='movieId')